In [ ]:
import os
from util import load_pfm

from PIL import Image
import numpy as np
from scipy.ndimage import filters

In [ ]:
DATASET_FOLDER = './MiddEval3/trainingQ'

In [ ]:
#%%time

def im2col_sliding_broadcasting(A, BSZ, stepsize=1):
    # Parameters
    M,N = A.shape
    col_extent = N - BSZ[1] + 1
    row_extent = M - BSZ[0] + 1

    # Get Starting block indices
    start_idx = np.arange(BSZ[0])[:,None]*N + np.arange(BSZ[1])

    # Get offsetted indices across the height and width of input array
    offset_idx = np.arange(row_extent)[:,None]*N + np.arange(col_extent)

    # Get all actual indices & index into input array for final output
    return np.take (A,start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize])

radius = 3
diameter = 2*radius+1
def census_transform(img,pairs):
    if False:
        img = np.sum(img,2)
        res = np.zeros((img.shape[0],img.shape[1],len(pairs)),np.int)
        l_comp = [p[0] for p in pairs]
        r_comp = [p[1] for p in pairs]
        l_comp_x = np.array([x[0] for x in l_comp])
        l_comp_y = np.array([x[1] for x in l_comp])
        r_comp_x = np.array([x[0] for x in r_comp])
        r_comp_y = np.array([x[1] for x in r_comp])
        for y in xrange(radius,img.shape[0]-radius-1):
            for x in xrange(radius,img.shape[1]-radius-1):
                p1 = img[l_comp_y + y,l_comp_x + x]
                p2 = img[r_comp_y + y,r_comp_x + x]
                res[y,x,:] = p1 > p2
        return res
    imgp = np.pad(np.sum(img,2),radius,'edge')
    i2m = im2col(imgp,[diameter,diameter])
    p12 = i2m > i2m[(diameter*diameter-1)/2,:]
    cen = p12[pairs,:].T
    cen = cen.reshape([img.shape[0],img.shape[1],len(pairs)])
    return cen
def census_match(left,right,max_disp,c_pairs):
    lc = census_transform(left,c_pairs)
    rc = census_transform(right,c_pairs)
    
    x_idx = np.array([(x-np.arange(max_disp)).clip(0) for x in xrange(lc.shape[1])])
    i_idx = np.array([ x-np.zeros(max_disp,dtype=np.int) for x in xrange(lc.shape[1])])
    lc_pad = lc[:,i_idx,:]
    rc_pad = rc[:,x_idx,:]
    costs = np.sum(lc_pad != rc_pad,3)

    return costs

#full_pairs = [((0,0),(int(i/diameter)-radius,i%diameter-radius)) for i in range(diameter*diameter)]
#pairs = [x for idx,x in enumerate(full_pairs) if idx%2==0 and x != ((0,0),(0,0))]
pairs = [i for i in range(diameter*diameter) if i%2 == 0 and i != (diameter*diameter-1)/2]
for image in os.listdir(DATASET_FOLDER): #['Motorcycle']:#
    fold_path = os.path.join(DATASET_FOLDER,image)
    gt_file = os.path.join(fold_path,'disp0GT.pfm')
    l_file = os.path.join(fold_path,'im0.png')
    r_file = os.path.join(fold_path,'im1.png')
    mask_file = os.path.join(fold_path,'mask0nocc.png')
    calib_file = os.path.join(fold_path,'calib.txt')

    if os.path.exists(gt_file):
        l = np.array(Image.open(l_file))
        r = np.array(Image.open(r_file))
        gt = load_pfm(gt_file)[0]
        mask = np.array(Image.open(mask_file))
        with open(calib_file) as fp:
            calib = {x.split('=')[0]:'='.join(x.split('=')[1:]) for x in fp.readlines()}
        if True:
            costs = census_match(l,r,int(calib['ndisp']),pairs)
            box_costs = filters.uniform_filter(costs,7)
            best = np.argmin(box_costs,2)
            plt.imshow(best)
            plt.figure()
            plt.imshow(gt)
            plt.show()  

In [ ]:
%%time
costs = census_match(l,r,int(calib['ndisp']),pairs)
box_costs = filters.uniform_filter(costs,7)
best = np.argmin(box_costs,2)
plt.imshow(best)
plt.show()